In [3]:

"""

ESN Implementation imported from: https://github.com/cknd/pyESN/blob/master/pyESN.py

Initialisation of reservoir weights and input weights done according to Jaeger's paper,
and implementation of augmented training algorithm following the paper:
    https://papers.nips.cc/paper/2318-adaptive-nonlinear-system-identification-with-echo-state-networks.pdf  

Readout training with Moore-Penrose Matrix Inverse or Ridge Regression (added)

Changes for Otsuka's model architecture:
    - sigmoid function for update of memory layer (reservoir units): 
        gain=4, added bias for the reservoir activations
    - default out activation: identity
    - get_states method to work with continuation
    - fit and predict methods take reservoir units (output from get_states) directly
    - RLS regression for online learning
    
Intrinsic Motivation Schmidhuber
    - history object: for each n in N_T (episodes*num_steps): [state, action, state2, reward]
    - history is evaluated on the last <= 10000 time-steps
    - calculateReward methods

"""

import numpy as np
from sklearn.linear_model import Ridge
from cannon_rlsfilter import RLSFilterAnalyticIntercept
import math
#import warnings
#warnings.filterwarnings("error")



def correct_dimensions(s, targetlength):
    """checks the dimensionality of some numeric argument s, broadcasts it
       to the specified length if possible.
    Args:
        s: None, scalar or 1D array
        targetlength: expected length of s
    Returns:
        None if s is None, else numpy vector of length targetlength
    """
    if s is not None:
        s = np.array(s)
        if s.ndim == 0:
            s = np.array([s] * targetlength)
        elif s.ndim == 1:
            if not len(s) == targetlength:
                raise ValueError("arg must have length " + str(targetlength))
        else:
            raise ValueError("Invalid argument")
    return s


# sigmoid
def sigmoid(x, gain=1):
    return 1 / (1 + np.exp(-gain*x))

# !!! do I apply the gain to the predictions as well?
def inv_sigmoid(x, gain=1):
    return np.log( (x*gain) / (1 - (x*gain) ) )

def atanh(x):
    #x is of shape (1, teachers)
    
    atanhx = np.zeros(x.shape[1])
    for i,v in enumerate(x[0]):
        atanhx[i] = math.atanh(v)
        
    return atanhx
        

def identity(x):
    return x

class ESN():

    def __init__(self, n_inputs, n_outputs, n_reservoir=200,
                 spectral_radius=0.95, sparsity=0,
                 noise=0.001,
                 readout='pseudo-inverse',
                 ridge_reg=None,
                 input_weights_scaling = 1,
                 input_scaling=None,input_shift=None,teacher_forcing=None, feedback_scaling=None,
                 teacher_scaling=None, teacher_shift=None,
                 out_activation=np.tanh, inverse_out_activation=atanh,
                 silent=True, 
                 augmented=False,
                 transient=200,
                 input_bias=0
                 ):
        """
        Args:
            n_inputs: nr of input dimensions
            n_outputs: nr of output dimensions
            n_reservoir: nr of reservoir neurons
            spectral_radius: spectral radius of the recurrent weight matrix
            sparsity: proportion of recurrent weights set to zero
            noise: noise added to each neuron (regularization)
            readout: type of readout 0 can be moonrose pseudo-inverse or ridge regression
            ridge_reg: regularisation value alpha if readout is Ridge
            
            input_weights_scaling: scaling of the input connection weights
            input_shift: scalar or vector of length n_inputs to add to each
                        input dimension before feeding it to the network.                       
            input_scaling: scalar or vector of length n_inputs to multiply
                        with each input dimension before feeding it to the netw.
                        
            teacher_shift: additive term applied to the target signal
            teacher_scaling: factor applied to the target signal
            teacher_forcing: if True, feed the target back into output units

            out_activation: output activation function (applied to the readout)
            inverse_out_activation: inverse of the output activation function
    
            silent: supress messages
            augmented: if True, use augmented training algorithm
            transient: how many initial states to discard
            
        """
        # check for proper dimensionality of all arguments and write them down.
        self.n_inputs = n_inputs
        self.n_reservoir = n_reservoir
        self.n_outputs = n_outputs   # part will be obs, part will be reward
        self.spectral_radius = spectral_radius
        self.sparsity = sparsity
        self.noise = noise
        self.readout = readout
        self.ridge_reg = ridge_reg
        
        self.input_weights_scaling = input_weights_scaling
        self.input_shift = correct_dimensions(input_shift, n_inputs)
        self.input_scaling = correct_dimensions(input_scaling, n_inputs)

        self.teacher_shift = teacher_shift
        self.teacher_scaling = teacher_scaling
        self.teacher_forcing = teacher_forcing

        self.out_activation = out_activation
        self.inverse_out_activation = inverse_out_activation

        self.silent = silent
        self.augmented = augmented
        self.transient = transient
        
        self.input_bias = input_bias
        
        self.laststate = np.zeros(self.n_reservoir)
        self.lastextendedstate = np.zeros(self.n_reservoir+self.n_inputs)
        self.lastinput = np.zeros(self.n_inputs+self.input_bias)
        self.lastoutput = np.zeros(self.n_inputs)
        
        
        
        self.defaultHistEval = 10000    # look at last n time steps when evaluating history
        
        self.initweights()
        
    def initweights(self):
        
        # initialize recurrent weights:
        self.W = self.initialise_reservoir()
        
        # bias for the update of the states of the reservoir
        self.reservoir_bias = np.dot(np.repeat(-0.5,self.n_reservoir),self.W)
            
        # [nk] following Jaeger's paper:
        # added scaling
        self.W_in = np.random.uniform(low = -0.1, high = 0.1, size = (self.n_reservoir, self.n_inputs+self.input_bias))*self.input_weights_scaling
             
        # random feedback (teacher forcing) weights:
        self.W_feedb = np.random.RandomState().rand(
            self.n_reservoir, self.n_outputs) * 2 - 1
                
        # filter for online learning
        self.RLSfilter = RLSFilterAnalyticIntercept(self.n_reservoir+self.n_inputs+self.input_bias, self.n_outputs, forgetting_factor=0.995)
          
        
    def initialise_reservoir(self):
        
        # [nk] following Jaeger's paper:
        W = np.random.uniform(low = -1, high = 1, size = (self.n_reservoir, self.n_reservoir))
        # delete the fraction of connections given by (self.sparsity):
        W[np.random.RandomState().rand(*W.shape) < self.sparsity] = 0
        # compute the spectral radius of these weights:
        radius = np.max(np.abs(np.linalg.eigvals(W)))
        # rescale them to reach the requested spectral radius:
        # if radius = 0, reinitialise weights again randomly
        try:
            W = W * (self.spectral_radius / radius)
        except:
            self.initialise_reservoir()
              
        return W
    
    def resetState(self):
        self.laststate = np.zeros(self.n_reservoir)
        self.lastextendedstate = np.zeros(self.n_reservoir+self.n_inputs)
        self.lastinput = np.zeros(self.n_inputs)
        self.lastoutput = np.zeros(self.n_inputs)

    def _update(self, state, input_pattern, output_pattern=None):
        """performs one update step.
        i.e., computes the next network state by applying the recurrent weights
        to the last state & and feeding in the current input and output patterns
        """
        
#         pdb.set_trace()
        
        if self.teacher_forcing:
            preactivation = (np.dot(self.W, state)
                             + np.dot(self.W_in, input_pattern)
                             + np.dot(self.W_feedb, output_pattern)
                             )
        else:
            preactivation = (np.dot(self.W, state)
                             + np.dot(self.W_in, input_pattern)
                             )
            
        # [nk] add noise - the original code added noise after applying non-linearity!
        preactivation = preactivation + self.noise * (np.random.uniform(0,1,self.n_reservoir))
        
        # apply activation function to the reservoir with the necessary gain and the bias = -0.5*W
        activation = sigmoid(preactivation,4) + self.reservoir_bias
        
        return activation

    def _scale_inputs(self, inputs):
        """for each input dimension j: multiplies by the j'th entry in the
        input_scaling argument, then adds the j'th entry of the input_shift
        argument."""
        if self.input_scaling is not None:
            inputs = np.dot(inputs, np.diag(self.input_scaling))
        if self.input_shift is not None:
            inputs = inputs + self.input_shift
        return inputs

    def _scale_teacher(self, teacher):
        """multiplies the teacher/target signal by the teacher_scaling argument,
        then adds the teacher_shift argument to it."""
        if self.teacher_scaling is not None:
            teacher = teacher * self.teacher_scaling
        if self.teacher_shift is not None:
            teacher = teacher + self.teacher_shift
        return teacher

    def _unscale_teacher(self, teacher_scaled):
        """inverse operation of the _scale_teacher method."""
        if self.teacher_shift is not None:
            teacher_scaled = teacher_scaled - self.teacher_shift
        if self.teacher_scaling is not None:
            teacher_scaled = teacher_scaled / self.teacher_scaling
        return teacher_scaled
    
    
    def get_states(self, inputs, extended, continuation, outputs=None, inspect=False):
        """
        [nk]
        Collect the network's neuron activations.
        Args:
            inputs: array of dimensions (N_training_samples x n_inputs)
            outputs: array of dimension (N_training_samples x n_outputs)
            inspect: show a visualisation of the collected reservoir states
        Returns:
            the network's states for every input sample
        """
        # transform any vectors of shape (x,) into vectors of shape (x,1):
        if inputs.ndim < 2:
            inputs = np.reshape(inputs, (len(inputs), -1))
        
        if outputs is not None and outputs.ndim < 2:
            outputs = np.reshape(outputs, (len(outputs), -1))

        n_samples = inputs.shape[0]
        
        # add bias to inputs if there is such
        if self.input_bias != 0:
            inputs = np.hstack((inputs, np.ones((n_samples,1))))
            
        # use last state, input, output
        if continuation:
            laststate = self.laststate
            lastinput = self.lastinput
            lastoutput = self.lastoutput
        else:
            laststate = np.zeros(self.n_reservoir)
            lastinput = np.zeros(self.n_inputs+self.input_bias)
            lastoutput = np.zeros(self.n_outputs)
            
        if not self.silent:
            print("harvesting states...")    

        # create scaled input and output vector
        inputs_scaled = np.vstack([lastinput, self._scale_inputs(inputs)])
        if outputs is not None:
            teachers_scaled = np.vstack([lastoutput, self._scale_teacher(outputs)])
        # create states vector
        states = np.vstack(
            [laststate, np.zeros((n_samples, self.n_reservoir))])
        
        
        if self.augmented:
            # create extended states vector
            lastaugmentedstate = np.hstack((np.hstack((lastinput, laststate)),
                                                        np.hstack((np.power(lastinput,2),np.power(laststate,2)))
                                                        ))
            augmented_states = np.vstack(
                    [lastaugmentedstate,np.zeros((n_samples, self.n_reservoir*2+2))])
            
        
        # activate the reservoir with the given input:
        for n in range(1, n_samples+1):
            if outputs is not None:
                states[n, :] = self._update(states[n - 1], inputs_scaled[n, :],
                                        teachers_scaled[n - 1, :])
            else:
                states[n, :] = self._update(states[n - 1], inputs_scaled[n, :])
            
            if self.augmented:
                # x_squares(n) =  (u(n), x1(n), ... , xN(n), u^2(n), x1^2(n), ... , xN^2(n))
                # ! teacher forcing version missing
                augmented_states[n,:] = np.hstack((np.hstack((inputs_scaled[n,:],states[n,:])),
                                                        np.hstack((np.power(inputs_scaled[n,:],2),np.power(states[n,:],2)))
                                                        ))
        # include the raw inputs for states
        extended_states = np.hstack((inputs_scaled, states))
        
        # remember the last state, input, output for later:
        self.laststate = states[-1, :]
        self.lastextendedstate = extended_states[-1,:]
        self.lastinput = inputs_scaled[-1, :]
        if outputs is not None:
            self.lastoutput = teachers_scaled[-1, :]
        
        # output states
        if self.augmented:
            out = augmented_states
        elif extended:
            out = extended_states
        else:
            out = states
       
        return out[1:]    #output without last state

    def fit(self, outputs, inputs, continuation, inspect=False):
        """
        [nk]
        Collect the network's reaction to training data, train readout weights.
        Args:
            inputs: array of dimensions (N_training_samples x n_inputs)
            outputs: array of dimension (N_training_samples x n_outputs)
            inspect: show a visualisation of the collected reservoir states
        Returns:
            the network's output on the training data, using the trained weights
        """
        # transform any vectors of shape (x,) into vectors of shape (x,1):
        if outputs.ndim < 2:
            outputs = np.reshape(outputs, (len(outputs), -1))
        # transform teacher signal:
        teachers_scaled = self._scale_teacher(outputs)
        
        # [nk] collect reservoir states
        states = self.get_states(inputs, extended=True, continuation=continuation)

        # learn the weights, i.e. find the linear combination of collected
        # network states that is closest to the target output
        if not self.silent:
            print("fitting...")
        
        # Solve for W_out:
        if self.readout == 'pseudo-inverse':
            self.W_out = np.dot(np.linalg.pinv(states[self.transient:, :]),
                        self.inverse_out_activation(teachers_scaled[self.transient:, :])).T
        elif self.readout == 'ridge':
            self.readout = Ridge(alpha=self.ridge_reg)
            self.readout.fit(states[self.transient:, :], teachers_scaled[self.transient:, :])
        else:
            raise ValueError('Invalid readout parameter: Must be either "ridge" or "pseudo-inverse".')

        # optionally visualize the collected states
        if inspect:
            from matplotlib import pyplot as plt
            # (^-- we depend on matplotlib only if this option is used)
            plt.figure(
                figsize=(states.shape[0] * 0.0025, states.shape[1] * 0.01))
            plt.imshow(states.T, aspect='auto',
                       interpolation='nearest')
            plt.colorbar()

        # apply learned weights to the collected states:
        if not self.silent:
            print("training (squared mean squared) error:")
        if self.readout == 'pseudo-inverse': 
            pred_train = self._unscale_teacher(self.out_activation(
                    np.dot(states, self.W_out.T)))
        else:   #ridge
            pred_train = self._unscale_teacher(self.readout.predict(states))
        if not self.silent:
            print(np.sqrt(np.mean((pred_train - outputs)**2)))
        return pred_train

    
    def predict(self, states):
        """
        Apply the learned weights to the network's reactions to new input.
        Args:
            states: the reservoir of the network which has been activated by the input
        Returns:
            Array of output activations
        """
        n_samples = states.shape[0]
          
        # output predictions for each input
        outputs = np.zeros((n_samples, self.n_outputs))
        for n in range(n_samples):
            if self.readout == 'pseudo-inverse':
                outputs[n, :] = self.out_activation(np.dot(self.W_out,states[n,:]))
            else:   # ridge
                outputs[n, :] = self.readout.predict(states[n,:].reshape(1,-1))
        
        unscaled_outputs = self._unscale_teacher(outputs)
        
        return unscaled_outputs 
    
    # array to store all obs + rewards
    def setHistory(self, episodes, steps):
        num_elem_hist = self.n_inputs + self.n_outputs
        self.history = np.repeat(-1, episodes*steps*num_elem_hist).reshape(episodes, steps, num_elem_hist)
    
    # internal reward: evaluate the current network on the history, fit it to the last teacher output from the history,
    # evaluate the new network, return difference between the two
    # Schmidhuber --> int_reward = C(p_old, hist) - C(p_new, hist)
    def calculateInternalReward(self, allEpisodes=False):
            
        #------ calc C(p_old, hist)
        num_elem_hist = self.n_inputs + self.n_outputs
        hist = self.history[self.history != -1]  # take all time steps that happened
        hist = hist.reshape(int(len(hist)/num_elem_hist),num_elem_hist) # reshape into (all time steps, hist elements)
        
        # take all history or last 10k times steps
        if allEpisodes or hist.shape[0] <= self.defaultHistEval:
            inputs = hist[:,:self.n_inputs]
            teachers = hist[:,self.n_inputs:]
        else:
            inputs = hist[-self.defaultHistEval:,:self.n_inputs]
            teachers = hist[-self.defaultHistEval:,self.n_inputs:]
        
        # apply inverse ou activation to the teacher signal
        teachers = self.inverse_out_activation(teachers)
        
        # get reservoir activations for all history
        res_states = self.get_states(inputs, extended=True, continuation=False)  #continuation is False because starts from first state
        
        # get predictions by applying the rls filter (without applying activation function)
        preds1 = np.zeros((inputs.shape[0], self.n_outputs))
        for i in range(inputs.shape[0]):
            preds1[i,:] = self.out_activation(self.RLSfilter.predict(res_states[i,:].reshape(-1,1))).T       
       
        # calculate predictor quality
        quality1 = np.sqrt(np.mean((preds1 - teachers)**2))
        
        #--------- update filter with last input-output
        self.RLSfilter.process_datum(res_states[-1,:].reshape(-1,1), teachers[-1,:].reshape(-1,1))
        
        #------- calc C(p_new, hist)
        preds2 = np.zeros((inputs.shape[0], self.n_outputs))
        for i in range(inputs.shape[0]):
            preds2[i,:] = self.RLSfilter.predict(res_states[i,:].reshape(-1,1)).T     
       
        # calculate predictor quality and save it
        self.quality = np.sqrt(np.mean((preds2 - teachers)**2))

        
        return (quality1 - self.quality)
    
        


In [29]:
"""
Code for ising taken from Aguilera's code for "Adaptation to Criticality":
Changes:
- Restricted - connections within hidden layer not being updated
- Embodiment for environment Frozen Lake
- RL Learning by a variant of SARSA - to Hinton's paper "Using Q-energies"
     *using sensors and motors variables (not the state variable)
     
For Otsuka's model architecture:
- Predictor ESN as parameter so it can have access to its reservoir and history
- Memory layer: doesn't get reset throughout training
- Inputs are binarised observation, memory, reward
- An episode is T*10 time steps

Intrinsic Motivation:
- internal reward is 0 to 1

""" 


import numpy as np
from itertools import combinations
import matplotlib.pyplot as plt
import gym
import time

import pdb


class ising:
    # Initialize the network
    def __init__(self, netsize, Nmemory, Nreward, Nsensors=1, Nmotors=1, predictor=None):  # Create ising model

#         pdb.set_trace()
        
        self.size = netsize		#Network size
        self.Ssize = Nsensors  # Number of sensors
        self.Msize = Nmotors  # Number of motors
        self.Memsize = Nmemory # Number of memory units
        self.Rewsize = Nreward # Number of units encoding reward
        self.Inpsize = Nsensors + Nmemory + Nreward  # Number of sensors+memory+reward

        self.h = np.zeros(netsize) # local biases
        self.J = np.zeros((netsize, netsize)) # symmetic weights between hidden variables
        self.max_weights = 2          # prevent (one of) the weights from growing too much

        if predictor is not None:
            self.predictor = predictor

        self.env = gym.make('FrozenLake8x8-v0')
        self.observation = self.env.reset()
        self.maxobs = 64   # !!!!!! For frozen Lake
        self.maxact = 4
        self.maxrew = 3
        
        self.randomize_state() # randomise states s
        self.initialise_sensors() # initialise sensors to point to start observation and to empty memory of the predictor

        self.BetaCrit = 1.0  
        self.defaultT = max(100, netsize * 20)

        self.Ssize1 = 0
        
        self.rewardsPerEpisode = 0     #keep track of rewards (for crit)
        self.successfulEpisodes = 0
        
    def get_state(self, mode='all'):
        if mode == 'all':
            return self.s
        elif mode == 'motors':
            return self.motors
        elif mode == 'sensors': 
            return self.s[0:self.Inpsize]
        elif mode == 'input':
            return self.sensors
        elif mode == 'non-sensors':
            return self.s[self.Inpsize:]
        elif mode == 'hidden':
            return self.s[self.Inpsize:-self.Msize]

    # gets the index of the configuration of the neurons - state as one number
    def get_state_index(self, mode='all'):
        return bool2int(0.5 * (self.get_state(mode) + 1))

    def initialise_sensors(self):
        self.sensors = np.repeat(-1, self.Ssize+self.Memsize+self.Rewsize)
        self.sensors[:self.Ssize] = bitfield(self.InputToIndex(self.observation, self.maxobs, self.Ssize), self.Ssize) * 2 - 1
    
    # Randomize the state of the network
    def randomize_state(self):     
        self.s = np.random.randint(0, 2, self.size) * 2 - 1           # make units -1 or 1     
    
    # Randomize the position of the agent
    def randomize_position(self):
        self.observation = self.env.reset()

    # Set random bias to sets of units of the system
    # bias to hidden and action
    def random_fields(self, max_weights=None):
        if max_weights is None:
            max_weights = self.max_weights
        self.h[self.Inpsize:] = max_weights * \
            (np.random.rand(self.size - self.Inpsize) * 2 - 1)

    # Set random connections to sets of units of the system
    def random_wiring(self, max_weights=None):  # Set random values for J
        if max_weights is None:
            max_weights = self.max_weights
        for i in range(self.size):
            for j in np.arange(i + 1, self.size):
                if i < j and (i >= self.Inpsize or j >= self.Inpsize):  # don't add connections between the sensors
                    # what about the motors? connection between the two will be added: doesn't matter because the correlations are not considered?
                    self.J[i, j] = (np.random.rand(1) * 2 - 1) * self.max_weights
        # get rid of connections between hidden units, and also motor (still dont know why this hasnt been done)
        self.J[self.Inpsize:-self.Msize,self.Inpsize:-self.Msize] = 0  # between hidden
        self.J[:self.Inpsize,-self.Msize:] = 0             # between sensor and motor
        self.J[-self.Msize:,-self.Msize:] = 0            # between motor

    # Update the position of the agent
    def Move(self):
        
#         pdb.set_trace()
        
        # step in environment and collect state2 and ext_reward
        action = int(np.digitize(np.sum(self.s[-self.Msize:]) / self.Msize, [-1.1, -1/2, 0, 1/2, 1.1])) - 1
        state2, ext_reward, done, info = self.env.step(action)
        
        # update predictor's history and collect its memory
        self.predictor.history[self.episode,self.t,:] = np.array([self.observation, action, state2, ext_reward])  # update predictor history
        # m' = f(s, a)  
        memory = self.predictor.get_states(np.array([self.observation, action]).reshape(1,-1), extended=False, continuation=True)
        
        # calculate internal reward
        int_reward = np.tanh(self.predictor.calculateInternalReward())
        if int_reward < 0:
            int_reward = 0
        reward = int_reward+ext_reward
        
        # new state = state2 + memory (from state,action) + reward (int + ext)
        self.observation = state2  
        self.UpdateSensors(state2,memory,reward) 
        
#        self.rewardsPerEpisode += reward    #update rewards per episode
#        self.observations[(self.observations == -1).argmax()] = observation      #add to list woth visited states
    
    # Transorm the sensor/motor input into integer index
    def InputToIndex(self, x, xmax, bitsize):
        return int(np.floor((x + xmax) / (2 * xmax + 10 * np.finfo(float).eps) * 2**bitsize))
    
    # Update the state of the sensor
    def UpdateSensors(self, state, memory, reward):  
        
        # binarise observation
        state_bit = bitfield(self.InputToIndex(state, self.maxobs, self.Ssize), self.Ssize) * 2 - 1 
        self.sensors[:self.Ssize] = state_bit
        # binarise memory
        memory_bit = np.zeros((1, memory.shape[1]))
        for i in range(memory.shape[1]):
            memory_bit[0,i] = -1 if memory[0,i] < 0.5 else 1
        self.sensors[self.Ssize:(self.Ssize+self.Memsize)]= memory_bit
        # binarise reward
        reward_bit = bitfield(self.InputToIndex(reward, self.maxrew, self.Rewsize), self.Rewsize) * 2 - 1
        self.sensors[(self.Ssize+self.Memsize):self.Inpsize]= reward_bit
        
    # Updates only the observation of the sensor, the memory and reward are kept the same
    def resetSensors(self, state):  
        
        state_bit = bitfield(self.InputToIndex(state, self.maxobs, self.Ssize), self.Ssize) * 2 - 1 
        self.sensors[:self.Ssize] = state_bit

    # Execute step of the Glauber algorithm to update the state of one unit
    def GlauberStep(self, i=None): 
        if i is None:
            i = np.random.randint(self.size)

        I = 0
        if i < self.Inpsize:
            I = self.sensors[i]
        eDiff = 2 * self.s[i] * (self.h[i] + I +
                                 np.dot(self.J[i, :] + self.J[:, i], self.s))
        if eDiff * self.BetaCrit < np.log(1 / np.random.rand() - 1):    # Glauber
            self.s[i] = -self.s[i]

    # Update random unit of the agent
    def Update(self, i=None):
        if i is None:
            i = np.random.randint(-1, self.size)
        if i == -1:
            self.Move()
        else:
            self.GlauberStep(i)

    # Sequentially update state of all units of the agent in random order
    def SequentialUpdate(self):
        for i in np.random.permutation(range(-1, self.size)):
            self.Update(i)

    # Step of the learning algorith to ajust correlations to the critical regime
    def AdjustCorrelations(self, T=None):
          
        if T is None:
            T = self.defaultT

        self.m = np.zeros(self.size)
        self.c = np.zeros((self.size, self.size))
        self.C = np.zeros((self.size, self.size))

        # Main simulation loop:
        self.x = np.zeros(T)      # to store the positions of the car during all T
        for t in range(T):
            
            self.SequentialUpdate()
            self.x[t] = self.observation
            self.m += self.s
            for i in range(self.size):
                self.c[i, i + 1:] += self.s[i] * self.s[i + 1:]
            
            self.t += 1
            
        self.m /= T
        self.c /= T
        for i in range(self.size):
            self.C[i, i + 1:] = self.c[i, i + 1:] - self.m[i] * self.m[i + 1:]

        c1 = np.zeros((self.size, self.size))
        for i in range(self.size):
            inds = np.array([], int)
            c = np.array([])
            for j in range(self.size):
                if not i == j:
                    inds = np.append(inds, [j])
                if i < j:
                    c = np.append(c, [self.c[i, j]])
                elif i > j:
                    c = np.append(c, [self.c[j, i]])
            order = np.argsort(c)[::-1]
            c1[i, inds[order]] = self.Cint[i, :]
        self.c1 = np.triu(c1 + c1.T, 1)
        self.c1 *= 0.5

        self.m[0:self.Inpsize] = 0          
        self.m1[0:self.Inpsize] = 0     #sensors have objective mean 0 but in the paper they say it's all of the units but the sensors that have mean 0??
        self.c[0:self.Inpsize, 0:self.Inpsize] = 0    #set corr in between sensors to 0
        self.c[-self.Msize:, -self.Msize:] = 0    #set corr in between motors to 0
        self.c[0:self.Inpsize, -self.Msize:] = 0    #set corr between sensors and motors to 0
        self.c1[0:self.Inpsize, 0:self.Inpsize] = 0
        self.c1[-self.Msize:, -self.Msize:] = 0
        self.c1[0:self.Inpsize, -self.Msize:] = 0
        
        # make it restricted BM
        self.c[self.Inpsize:-self.Msize,self.Inpsize:-self.Msize] = 0   #set corr in between hidden units to 0
        self.c1[self.Inpsize:-self.Msize,self.Inpsize:-self.Msize] = 0   #for reference as well
        
        dh = self.m1 - self.m
        dJ = self.c1 - self.c

        return dh, dJ

    # Algorithm for poising an agent in a critical regime
    def CriticalLearning(self, Iterations, T=None):
        
#         pdb.set_trace()
        tic = time.perf_counter()
    
        self.observations = np.repeat(-1,Iterations*T)    #keep track of reached states
        self.predictor.setHistory(int(Iterations/10)+1, T*10 ) # resetting the env every 10 iterations, so episodes and steps adjusted accordingly 
        
        self.t = 0
#         self.it = 0
        self.episode = 0
        
        u = 0.01
        count = 0
        dh, dJ = self.AdjustCorrelations(T)
        for it in range(Iterations):
            
#             print(self.episode, self.t)
#             self.it += 1
#             self.t += self.it*T
            count += 1
            self.h += u * dh
            self.J += u * dJ

            if it % 10 == 0:
                
                toc = time.perf_counter()
                print('Episode ' + str(self.episode) + ' took ' + str(int((toc - tic)/60)) + ' minutes.')
                
                tic = time.perf_counter()
#                 pdb.set_trace()
                
                self.randomize_position()  # updates self.observation to be 0
                
                # update the sensors with observation=0, the memory and the reward are kept the same
                self.resetSensors(self.observation)
                self.randomize_state()
                
                self.episode += 1  # first episode will have T instead of T*10 entries but thats ok because the predictor's reward method only takes actual entries
                self.t = 0
#                 print('reset'+str(self.episode)+str(self.t))
#                 self.it = 0
                # !!!! different from original - it randomises the sensors as well.
                
#                if self.rewardsPerEpisode >= 1:     # keep track of the times the agent reached the goal
#                    self.successfulEpisodes += 1
#                self.rewardsPerEpisode = 0
                
            Vmax = self.max_weights
            for i in range(self.size):
                if np.abs(self.h[i]) > Vmax:        # why do we need to restrict the weights and biases to a maximum value?
                    self.h[i] = Vmax * np.sign(self.h[i])
                for j in np.arange(i + 1, self.size):
                    if np.abs(self.J[i, j]) > Vmax:
                        self.J[i, j] = Vmax * np.sign(self.J[i, j])

            dh, dJ = self.AdjustCorrelations(T)

        
        
# Transform bool array into positive integer
# [nk] encodes the state of the neurons as one number
def bool2int(x):
    y = 0
    for i, j in enumerate(np.array(x)[::-1]):
        y += j * 2**i
    return int(y)

# Transform positive integer into bit array
def bitfield(n, size):
    x = [int(x) for x in bin(int(n))[2:]]
    x = [0] * (size - len(x)) + x
    return np.array(x)


In [36]:
import numpy as np
import pdb
import os
import pickle

# esn params
predictor_inputs = 2 # linear state,action
predictor_outputs = 2 # linear state2, reward
predictor_reservoir = 12
predictor_radius = 0.9
predictor_sparsity = 0.9
out_act = identity
inv_out_act = identity

predictor = ESN(n_inputs = predictor_inputs, n_outputs = predictor_outputs, n_reservoir=predictor_reservoir,
                 spectral_radius=predictor_radius, sparsity=predictor_sparsity,
                 out_activation=out_act, inverse_out_activation=inv_out_act,
                input_bias = 1
                 )

# rbm params
Ssize=7
Msize=3
Nhidden = 30 #more hidden due to more visible units: 23 sensors and 4 motors    #20 
Nmemory = predictor_reservoir
Nreward = 4
size=Ssize+Nmemory+Nhidden+Nreward+Msize

#actor
I = ising(size,Nmemory,Nreward,Ssize,Msize,predictor)

# Import reference correlations
filecorr = 'correlations-ising2D-size400.npy'
Cdist = np.load(filecorr)

# reorder reference correlations to match network's current correlations
I.m1 = np.zeros(size)
I.Cint = np.zeros((size, size - 1))
for i in range(size):
    c = []
    for j in range(size - 1):
        ind = np.random.randint(len(Cdist))
        c += [Cdist[ind]]
    I.Cint[i, :] = -np.sort(-np.array(c))
    
# critical
Iterations = 10000                          # originally 10000
T = 1000                                    # originally 5000

I.CriticalLearning(Iterations, T)

# Params to save
params = {}
params['Ssize']= Ssize
params['Msize']= Msize
params['nhidden'] = Nhidden
params['size'] = size
params['Iterations'] = Iterations
params['T'] = T
params['network_J'] = I.J
params['network_h'] = I.h

# Name of the file to save run

filetosave = 'critical-rbm+esn.pkl'

# Save
if not os.path.isfile(filetosave):
    with open(filetosave, 'wb') as output:
            pickle.dump(params, output, pickle.HIGHEST_PROTOCOL)
else:
    print("File already exists!")

Episode 0 took 0 minutes.
Episode 1 took 0 minutes.
Episode 2 took 0 minutes.
Episode 3 took 0 minutes.
Episode 4 took 0 minutes.


In [28]:
I.predictor.history

array([[[ 0,  0,  0,  0],
        [ 0,  1,  0,  0],
        [ 0,  2,  0,  0],
        ...,
        [-1, -1, -1, -1],
        [-1, -1, -1, -1],
        [-1, -1, -1, -1]],

       [[ 0,  2,  0,  0],
        [ 0,  1,  0,  0],
        [ 0,  1,  0,  0],
        ...,
        [19,  1, 19,  0],
        [19,  1, 19,  0],
        [19,  0, 19,  0]],

       [[ 0,  1,  1,  0],
        [ 1,  0,  1,  0],
        [ 1,  0,  1,  0],
        ...,
        [41,  1, 41,  0],
        [41,  2, 41,  0],
        [41,  0, 41,  0]],

       [[ 0,  1,  0,  0],
        [ 0,  1,  8,  0],
        [ 8,  0,  8,  0],
        ...,
        [35,  0, 35,  0],
        [35,  0, 35,  0],
        [35,  2, 35,  0]],

       [[ 0,  3,  0,  0],
        [ 0,  2,  8,  0],
        [ 8,  2,  0,  0],
        ...,
        [49,  3, 49,  0],
        [49,  3, 49,  0],
        [49,  3, 49,  0]],

       [[ 0,  2,  8,  0],
        [ 8,  0,  0,  0],
        [ 0,  0,  8,  0],
        ...,
        [41,  3, 41,  0],
        [41,  3, 41,  0],
  